In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [2]:
df_2 = pd.read_csv("prepared_data_sample_2.csv", parse_dates=["date"])

In [3]:
#Verschiedene Wachstumsraten 
df_2["d_close_pct"] = ( df_2["hdax_close"] - df_2["hdax_close"].shift(1) ) / df_2["hdax_close"].shift(1)
df_2["d_close_log"] = ( np.log(df_2["hdax_close"]) - np.log(df_2["hdax_close"].shift(1)))
df_2["d_close_diff"] = ( df_2["hdax_close"] - df_2["hdax_close"].shift(1) )

In [4]:
#Verschiedene Wachstumsraten für Total Cases
df_2["d_total_pct"] = ( df_2["total_cases"] - df_2["total_cases"].shift(1) ) / df_2["total_cases"].shift(1)
df_2["d_total_log"] = ( np.log(df_2["total_cases"]) - np.log(df_2["total_cases"].shift(1)))
df_2["d_total_diff"] = ( df_2["total_cases"] - df_2["total_cases"].shift(1) )

In [5]:
#Verschiedene Wachstumsraten für New Cases
df_2["d_new_pct"] = ( df_2["new_cases"] - df_2["new_cases"].shift(1) ) / df_2["new_cases"].shift(1)
df_2["d_new_log"] = ( np.log(df_2["new_cases"]) - np.log(df_2["new_cases"].shift(1)))
df_2["d_new_diff"] = ( df_2["new_cases"] - df_2["new_cases"].shift(1) )

In [6]:
#Verschiedene Wachstumsraten für Containment Index
df_2["d_containment_pct"] = ( df_2["containment_index"] - df_2["containment_index"].shift(1) ) / df_2["containment_index"].shift(1)
df_2["d_containment_log"] = ( np.log(df_2["containment_index"]) - np.log(df_2["containment_index"].shift(1)))
df_2["d_containment_diff"] = ( df_2["containment_index"] - df_2["containment_index"].shift(1) )

In [7]:
df_2.head(5)

,date,hdax_open,hdax_close,total_cases,new_cases,stringency_index,containment_index,Monday,Tuesday,Wednesday,...,d_close_diff,d_total_pct,d_total_log,d_total_diff,d_new_pct,d_new_log,d_new_diff,d_containment_pct,d_containment_log,d_containment_diff
0,2020-10-01,7051.18,7034.37,295539.0,2626.0,49.54,54.46,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-10-02,6990.41,7009.84,298374.0,2835.0,49.54,54.46,0,0,0,...,-24.53,0.009593,0.009547,2835.0,0.079589,0.076580,209.0,0.0,0.0,0.0
2,2020-10-05,7056.22,7092.98,304673.0,3100.0,49.54,54.46,1,0,0,...,83.14,0.021111,0.020891,6299.0,0.093474,0.089360,265.0,0.0,0.0,0.0
3,2020-10-06,7108.30,7124.25,307127.0,2454.0,49.54,54.46,0,1,0,...,31.27,0.008055,0.008022,2454.0,-0.208387,-0.233683,-646.0,0.0,0.0,0.0
4,2020-10-07,7125.75,7128.76,311137.0,4010.0,49.54,54.46,0,0,1,...,4.51,0.013056,0.012972,4010.0,0.634067,0.491072,1556.0,0.0,0.0,0.0


1. Bei Wachstumsraten ist das Problem, wenn keine new Cases und Nullen. Deswegen angefangen ab dem 25. Februar.
2. Man merkt, dass Wachstumsraten anfangs sehr hoch sind, deswegen log Wachstum sehr ungenau
--> Also kein log Wachstum für covid_cases und containment verwenden!

In [8]:
df_2.drop(columns=["hdax_open", "hdax_close", "total_cases",], inplace=True)

In [61]:
#ADF-Test um die verschiedenen Zeitreihen auf Stationarität zu prüfen
from statsmodels.tsa.stattools import adfuller


def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)

    
#KPSS-Test um die verschiedenen Zeitreiehen auf Stationarität zu prüfen  
from statsmodels.tsa.stattools import kpss


def kpss_test(timeseries):
    print("Results of KPSS Test:")
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = pd.Series(
        kpsstest[0:3], index=["Test Statistic", "p-value", "Lags Used"]
    )
    for key, value in kpsstest[3].items():
        kpss_output["Critical Value (%s)" % key] = value
    print(kpss_output)
    

# 1. HDAX Wachstumsraten auf Stationarität testen

## 1.1 ADF-Test

In [10]:
adf_test(df_2["d_close_pct"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                -8.439855e+00
p-value                        1.775296e-13
#Lags Used                     1.000000e+00
Number of Observations Used    1.850000e+02
Critical Value (1%)           -3.466201e+00
Critical Value (5%)           -2.877293e+00
Critical Value (10%)          -2.575168e+00
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

In [11]:
adf_test(df_2["d_close_log"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                -8.435567e+00
p-value                        1.820694e-13
#Lags Used                     1.000000e+00
Number of Observations Used    1.850000e+02
Critical Value (1%)           -3.466201e+00
Critical Value (5%)           -2.877293e+00
Critical Value (10%)          -2.575168e+00
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

In [12]:
adf_test(df_2["d_close_diff"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                -8.911455e+00
p-value                        1.101457e-14
#Lags Used                     1.000000e+00
Number of Observations Used    1.850000e+02
Critical Value (1%)           -3.466201e+00
Critical Value (5%)           -2.877293e+00
Critical Value (10%)          -2.575168e+00
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

## 1.2 KPSS-Test

In [13]:
kpss_test(df_2["d_close_pct"].dropna())

Results of KPSS Test:
Test Statistic           0.037257
p-value                  0.100000
Lags Used                1.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

In [14]:
kpss_test(df_2["d_close_log"].dropna())

Results of KPSS Test:
Test Statistic           0.036795
p-value                  0.100000
Lags Used                1.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

In [15]:
kpss_test(df_2["d_close_diff"].dropna())

Results of KPSS Test:
Test Statistic           0.033894
p-value                  0.100000
Lags Used                3.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.



# 2. Covid Cases auf Stationarität untersuchen

## 2.1 ADF-Test

In [16]:
adf_test(df_2["d_total_pct"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -1.929795
p-value                          0.318177
#Lags Used                       8.000000
Number of Observations Used    178.000000
Critical Value (1%)             -3.467632
Critical Value (5%)             -2.877918
Critical Value (10%)            -2.575501
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can not be rejected. Hence, the series is non-stationary.

In [17]:
adf_test(df_2["d_total_log"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -1.936746
p-value                          0.314977
#Lags Used                       8.000000
Number of Observations Used    178.000000
Critical Value (1%)             -3.467632
Critical Value (5%)             -2.877918
Critical Value (10%)            -2.575501
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can not be rejected. Hence, the series is non-stationary.

In [18]:
adf_test(df_2["d_total_diff"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -2.280726
p-value                          0.178259
#Lags Used                      14.000000
Number of Observations Used    172.000000
Critical Value (1%)             -3.468952
Critical Value (5%)             -2.878495
Critical Value (10%)            -2.575809
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can not be rejected. Hence, the series is non-stationary.

## 2.2 KPSS-Test

In [19]:
kpss_test(df_2["d_total_pct"].dropna())

Results of KPSS Test:
Test Statistic           1.401233
p-value                  0.010000
Lags Used                9.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is non-stationary as per the KPSS test.

In [20]:
kpss_test(df_2["d_total_log"].dropna())

Results of KPSS Test:
Test Statistic           1.408117
p-value                  0.010000
Lags Used                9.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is smaller than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is non-stationary as per the KPSS test.



In [21]:
kpss_test(df_2["d_total_diff"].dropna())

Results of KPSS Test:
Test Statistic           0.430057
p-value                  0.064200
Lags Used                9.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


Based upon the significance level of 0.05 and the p-value of KPSS test, there is evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is non-stationary as per the KPSS test.

# 3. New Cases auf Stationarität untersuchen

## 3.1 ADF-Test

In [22]:
adf_test(df_2["d_new_pct"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -3.986223
p-value                          0.001484
#Lags Used                       5.000000
Number of Observations Used    181.000000
Critical Value (1%)             -3.467005
Critical Value (5%)             -2.877644
Critical Value (10%)            -2.575355
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

In [23]:
adf_test(df_2["d_new_log"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -2.262179
p-value                          0.184456
#Lags Used                      12.000000
Number of Observations Used    174.000000
Critical Value (1%)             -3.468502
Critical Value (5%)             -2.878298
Critical Value (10%)            -2.575704
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can not be rejected. Hence, the series is non-stationary.

In [24]:
adf_test(df_2["d_new_diff"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                  -3.402967
p-value                          0.010858
#Lags Used                       8.000000
Number of Observations Used    178.000000
Critical Value (1%)             -3.467632
Critical Value (5%)             -2.877918
Critical Value (10%)            -2.575501
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

## 3.2 KPSS-Test

In [25]:
kpss_test(df_2["d_new_pct"].dropna())

Results of KPSS Test:
Test Statistic             0.328316
p-value                    0.100000
Lags Used                119.000000
Critical Value (10%)       0.347000
Critical Value (5%)        0.463000
Critical Value (2.5%)      0.574000
Critical Value (1%)        0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

In [26]:
kpss_test(df_2["d_new_log"].dropna())

Results of KPSS Test:
Test Statistic            0.371154
p-value                   0.089589
Lags Used                23.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

In [27]:
kpss_test(df_2["d_new_diff"].dropna())

Results of KPSS Test:
Test Statistic            0.164565
p-value                   0.100000
Lags Used                21.000000
Critical Value (10%)      0.347000
Critical Value (5%)       0.463000
Critical Value (2.5%)     0.574000
Critical Value (1%)       0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

# 4. Containment Index auf Stationarität prüfen

## 4.1 ADF-Test

In [28]:
adf_test(df_2["d_containment_pct"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                -1.209929e+01
p-value                        2.036010e-22
#Lags Used                     0.000000e+00
Number of Observations Used    1.860000e+02
Critical Value (1%)           -3.466005e+00
Critical Value (5%)           -2.877208e+00
Critical Value (10%)          -2.575122e+00
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

In [29]:
adf_test(df_2["d_containment_diff"].dropna())

Results of Dickey-Fuller Test:
Test Statistic                -1.204503e+01
p-value                        2.677309e-22
#Lags Used                     0.000000e+00
Number of Observations Used    1.860000e+02
Critical Value (1%)           -3.466005e+00
Critical Value (5%)           -2.877208e+00
Critical Value (10%)          -2.575122e+00
dtype: float64


Based upon the significance level of 0.05 and the p-value of ADF test, the null hypothesis can be rejected. Hence, the series is stationary.

## 4.2 KPSS-Test

In [30]:
kpss_test(df_2["d_containment_pct"].dropna())

Results of KPSS Test:
Test Statistic           0.19436
p-value                  0.10000
Lags Used                0.00000
Critical Value (10%)     0.34700
Critical Value (5%)      0.46300
Critical Value (2.5%)    0.57400
Critical Value (1%)      0.73900
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.

In [31]:
kpss_test(df_2["d_containment_diff"].dropna())

Results of KPSS Test:
Test Statistic           0.161665
p-value                  0.100000
Lags Used                0.000000
Critical Value (10%)     0.347000
Critical Value (5%)      0.463000
Critical Value (2.5%)    0.574000
Critical Value (1%)      0.739000
dtype: float64


/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(


Based upon the significance level of 0.05 and the p-value of KPSS test, there is no evidence for rejecting the null hypothesis in favor of the alternative. Hence, the series is stationary as per the KPSS test.



# 5 Zusammenfassung

In [43]:

list(df_2.loc[:, "d_close_pct": "d_containment_diff"].columns)

['d_close_pct',
 'd_close_log',
 'd_close_diff',
 'd_total_pct',
 'd_total_log',
 'd_total_diff',
 'd_new_pct',
 'd_new_log',
 'd_new_diff',
 'd_containment_pct',
 'd_containment_log',
 'd_containment_diff']

In [63]:
def kpss_stationarity(p_value):
    if p_value <= 0.05:
        return "Non-Stationary"
    else:
        return "Stationary"
    
    
def kpss_p_value(timeseries):
    kpsstest = kpss(timeseries, regression="c", nlags="auto")
    kpss_output = kpsstest[1]
    return kpss_output




def adf_stationarity(p_value):
    if p_value <= 0.05:
        return "Stationary"
    else:
        return "Non-Stationary"

def adf_p_value(timeseries):
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = dftest[1]
    return dfoutput


In [66]:
results = pd.DataFrame({"Timeseries": list(df_2.loc[:, "d_close_pct": "d_containment_diff"].columns),
                        "KPSS p-Value": [kpss_p_value(df_2[column].dropna()) for column in list(df_2.loc[:, "d_close_pct": "d_containment_diff"].columns)],
                        "ADF p-Value": [adf_p_value(df_2[column].dropna()) for column in list(df_2.loc[:, "d_close_pct": "d_containment_diff"].columns)]})

/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1910: InterpolationWarning: The test statistic is outside of the range of p-values available in the
look-up table. The actual p-value is greater than the p-value returned.

  warnings.warn(
/Users/maximbuz/Anaconda/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/stattools.py:1906: InterpolationWarning: The test statistic is outside of t

In [68]:
results["KPSS Stationarity"] = results["KPSS p-Value"].apply(kpss_stationarity)
results["ADF Stationarity"] = results["ADF p-Value"].apply(adf_stationarity)

In [69]:
results

,Timeseries,KPSS p-Value,ADF p-Value,KPSS Stationarity,ADF Stationarity
0,d_close_pct,0.100000,1.775296e-13,Stationary,Stationary
1,d_close_log,0.100000,1.820694e-13,Stationary,Stationary
2,d_close_diff,0.100000,1.101457e-14,Stationary,Stationary
3,d_total_pct,0.010000,3.181772e-01,Non-Stationary,Non-Stationary
4,d_total_log,0.010000,3.149767e-01,Non-Stationary,Non-Stationary
5,d_total_diff,0.064200,1.782595e-01,Stationary,Non-Stationary
6,d_new_pct,0.100000,1.484397e-03,Stationary,Stationary
7,d_new_log,0.089589,1.844562e-01,Stationary,Non-Stationary
8,d_new_diff,0.100000,1.085813e-02,Stationary,Stationary
9,d_containment_pct,0.100000,2.036010e-22,Stationary,Stationary
